<a href="https://colab.research.google.com/github/vikas-singh16/Semaantic-Search-For-Covid-19-Queries/blob/main/Semantic_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install faiss-cpu

     |████████████████████████████████| 8.6 MB 3.8 MB/s 


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 596 kB 51.8 MB/s 
     |████████████████████████████████| 6.6 MB 45.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoModel, AutoTokenizer

In [ ]:
token = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import json
file_handle = open("/content/drive/MyDrive/Semantic Search/data.json")
data = json.load(file_handle)
file_handle.close()

In [ ]:
documents_file = [doc["text"] for doc in data]
documents_file

['A pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.',
 'H

In [ ]:
tokenizer = token(data[0], return_tensors='pt')
tokenizer

{'input_ids': tensor([[  101,  1037,  6090,  3207,  7712,  2003,  2019, 16311,  1997,  2019,
         16514,  4295,  2008,  2038,  3659,  2408,  1037,  2312,  2555,  1010,
          2005,  6013,  3674, 17846,  2030,  4969,  1010, 12473,  1037,  6937,
          2193,  1997,  2111,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
model(**tokenizer)[0]

tensor([[[ 0.1025, -0.2056, -0.5713,  ..., -0.0051,  0.1253,  0.7564],
         [-0.3454, -0.0013, -0.7068,  ..., -0.3020,  0.0376,  0.4525],
         [ 0.0288, -0.1346, -0.0729,  ..., -0.2813,  0.3779, -0.0193],
         ...,
         [ 0.2545,  0.4376, -0.0790,  ..., -0.3760, -0.1083, -0.3239],
         [ 0.8220,  0.1464, -0.5341,  ...,  0.1889, -0.5492, -0.2128],
         [ 0.1956,  0.2456, -0.4352,  ..., -0.0896, -0.2934,  0.2751]]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
def encoder(document):

  return model(**token(text = document, truncation=True, return_tensors='pt'))[0].detach().squeeze()

In [ ]:
sentence_embeding = []
for doc in data:
  sentence_embeding.append(encoder(doc))

In [ ]:
for i in sentence_embeding:
  print(i.size())  

torch.Size([35, 768])
torch.Size([37, 768])
torch.Size([25, 768])
torch.Size([18, 768])
torch.Size([24, 768])
torch.Size([55, 768])
torch.Size([57, 768])
torch.Size([24, 768])
torch.Size([27, 768])
torch.Size([35, 768])
torch.Size([43, 768])


In [ ]:
import torch

In [ ]:
word2vec = []
for doc in sentence_embeding:
  word2vec.append(torch.mean(doc, axis = 0))

In [ ]:
for word in word2vec:
  print(word.size())

torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])
torch.Size([768])


In [ ]:
import faiss
import numpy as np

In [ ]:
index = faiss.IndexFlatIP(768)

In [ ]:
mapping = faiss.IndexIDMap(index )
mapping

<faiss.swigfaiss.IndexIDMap; proxy of <Swig Object of type 'faiss::IndexIDMapTemplate< faiss::Index > *' at 0x7fba509ada20> >

In [ ]:
mapping.add_with_ids(np.array([doc_vec.numpy() for doc_vec in word2vec]),np.arange(0,len(data)))

In [ ]:
len(word2vec)

11

In [ ]:
np.array([doc_vec.numpy() for doc_vec in word2vec])

array([[ 0.04860862,  0.09741875, -0.0493001 , ..., -0.35713515,
        -0.17343101,  0.3521632 ],
       [-0.12289891, -0.14739096, -0.27448386, ..., -0.15979874,
        -0.01640391,  0.22394593],
       [-0.1506394 , -0.00387476, -0.16962844, ..., -0.16288328,
        -0.05760337,  0.12173302],
       ...,
       [-0.06405506,  0.00731968, -0.22593707, ..., -0.28433204,
         0.06029751,  0.37737986],
       [ 0.02226315, -0.11046934, -0.24745277, ..., -0.07326985,
        -0.06600287,  0.29851404],
       [-0.20975545, -0.11904943,  0.03429305, ..., -0.10559369,
         0.1210614 ,  0.13563669]], dtype=float32)

In [ ]:
file_handle = open("/content/drive/MyDrive/Semantic Search/questions.json")
question = json.load(file_handle)
file_handle.close()

In [ ]:
ques = "what is a covid-19 ?"

In [ ]:
question

['How many people have died during Black Death?',
 'Which diseases can be transmitted by animals?',
 'Connection between climate change and a likelihood of a pandemic',
 'What is an example of a latent virus',
 'Viruses in nanotechnology',
 'Giant viruses classification',
 'What are the notable pandemic prevention organizations?',
 'How many leprosy outbreaks are known to happen?',
 'What are the geographic areas with the highest transmission of malaria?',
 'How to prevent the spread of viral infections?']

In [ ]:
encode_ques_0 = encoder(ques)

encoder_ques_0_mean = torch.mean(encode_ques_0,axis=0)

encoder_ques_0_mean.size()

torch.Size([768])

In [ ]:
encoder_query = encoder_ques_0_mean.unsqueeze(axis=0).numpy()

In [ ]:
knn_docs = mapping.search(encoder_query,2)

knn_docs

(array([[41.62216 , 41.281242]], dtype=float32), array([[9, 2]]))

In [ ]:
knn_docs[0][0]

array([41.62216 , 41.281242], dtype=float32)

In [ ]:
knn_docs[1][0]

array([9, 2])

In [ ]:
results = [data[idx] for idx in knn_docs[1][0]]
results

['The Spanish flu, also known as the 1918 flu pandemic, was an unusually deadly influenza pandemic caused by the H1N1 influenza A virus.',
 'Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.']

In [ ]:
def search(query, knn_search):
  encode = encoder(query)
  encoder_mean = torch.mean(encode, axis = 0)
  encoder_query = encoder_mean.unsqueeze(axis =0 ).numpy()
  knn_docs = mapping.search(encoder_query, knn_search)
  results = [data[idx] for idx in knn_docs[1][0]]
  similarity_score = knn_docs[0][0]

  return list(zip(results,similarity_score))
                            

In [ ]:
search(question[6],2) # results from IndexFlatL2

[('Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.', 29.367191),
 ('The COVID-19 pandemic, also known as the coronavirus pandemic, is an ongoing pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).',
  33.297684)]

In [ ]:
search(question[6],2) #Results from IndexFlatIP

[({'text': 'Pandemic prevention is the organization and management of preventive measures against pandemics. Those include measures to reduce causes of new infectious diseases and measures to prevent outbreaks and epidemics from becoming pandemics.\nIt is not to be mistaken for pandemic preparedness or pandemic mitigation which largely seek to mitigate the magnitude of negative effects of pandemics and may overlap with pandemic prevention in some respects.',
   'title': 'Pandemic prevention',
   'url': 'https://en.wikipedia.org/wiki/Pandemic_prevention'},
  51.70147),
 ({'text': 'Targeted immunization strategies are approaches designed to increase the immunization level of populations and decrease the chances of epidemic outbreaks. Though often in regards to use in healthcare practices and the administration of vaccines to prevent biological epidemic outbreaks, these strategies refer in general to immunization schemes in complex networks, biological, social or artificial in nature. Ide